In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
from datetime import datetime
import streamlit as st

In [57]:
def parse_german_date(date_string):
    if pd.isna(date_string):
        return pd.NaT
    try:
        german_months = {
            'Jan': 1, 'Feb': 2, 'Mär': 3, 'Apr': 4, 'Mai': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Okt': 10, 'Nov': 11, 'Dez': 12
        }
        day, month, year = date_string.split()
        month_num = german_months[month]
        return datetime(2000 + int(year), month_num, int(day))
    except (ValueError, KeyError):
        return pd.NaT  # Return Not-a-Time for any parsing errors

In [60]:
res = requests.get(f"https://www.eqs-news.com/wp/wp-admin/admin-ajax.php?lang=de&action=fetch_realtime_events_data&recordsFrom[0][api_type]=events&recordsFrom[0][category]=future&pageLimit=10000&pageNo=1&additional[is_new]=false&additional[mode]=append&loadFrom=mysql&time=")
if (res.status_code == 200):
    soup = BeautifulSoup(res.text, 'html.parser')
    events = []  
    event_divs = soup.find_all('div', class_='event')
    
    for event in event_divs:
        event_data = {}
        event_data['company'] = event.find('h4', class_='event__company').get_text(strip=True)
        event_data['title'] = event.find('p', class_='event__title').get_text(strip=True)
        #event_data['location'] = event.find('div', class_='event__location').get_text(strip=True) if event.find('div', class_='event__location') else None
        date = event.find('p', class_='event__date').get_text(strip=True)
        month_year = event.find('p', class_='event__month-year').get_text(strip=True)
        event_data['date'] = f"{date} {month_year}"
        #event_data['company_url'] = event['data-events-company-url']
        #event_data['company_share_url'] = event['data-events-company-share-url']
        #event_data['uuid'] = event['data-events-uuid']
        events.append(event_data)
    
    df = pd.DataFrame(events)
    df['converted_dates'] = df['date'].apply(parse_german_date)
    df.drop(columns=['date'], inplace=True)
    
df

,company,title,converted_dates
0,Intevac,Analystenveranstaltung,2024-08-05
1,WashTec AG,Veröffentlichung Halbjahresfinanzbericht,2024-08-05
2,Infineon Technologies AG,Veröffentlichung Quartalsmitteilung (Stichtag Q3),2024-08-05
3,Aurubis AG,Veröffentlichung Quartalsmitteilung (Stichtag Q3),2024-08-05
4,Crescent Energy Company,,2024-08-06
...,...,...,...
1077,IVU Traffic Technologies AG,Veröffentlichung Quartalsmitteilung (Stichtag Q3),2025-11-20
1078,SCHOTT Pharma AG & Co. KGaA,Veröffentlichung Jahresfinanzbericht,2025-12-10
1079,All for One Group SE,Analystenveranstaltung,2025-12-15
1080,All for One Group SE,Bilanzpressekonferenz,2025-12-15


In [13]:
df.to_excel("EqsLoaderPlayground.xlsx", index=False)

In [62]:
st.dataframe(df)

2024-08-05 21:43:18.770 
  command:

    streamlit run D:\zail\eqsloader\eqsloader\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()